In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import statistics
import re

import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Load data
dtset = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Preprocessing/nhaban_clean.csv")
dtset_columns = dtset.columns.tolist()
print("+ Shape: ", dtset.shape) # (Number of Lines, Number of Columns)
print("+ Column: ", dtset_columns) # Name the attribute columns.
dtset.info()

+ Shape:  (28658, 20)
+ Column:  ['Tiêu đề', 'Địa chỉ', 'Ngày đăng', 'Ngày hết hạn', 'Loại tin', 'Mã tin', 'Người đăng', 'Thông tin mô tả', 'Diện tích', 'Mức giá', 'Mặt tiền', 'Đường vào', 'Hướng ban công', 'Hướng nhà', 'Số tầng', 'Số phòng ngủ', 'Số toilet', 'Pháp lý', 'Nội thất', 'Loại hình']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28658 entries, 0 to 28657
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tiêu đề          28658 non-null  object 
 1   Địa chỉ          28658 non-null  object 
 2   Ngày đăng        28658 non-null  object 
 3   Ngày hết hạn     28658 non-null  object 
 4   Loại tin         28658 non-null  object 
 5   Mã tin           28658 non-null  int64  
 6   Người đăng       28658 non-null  object 
 7   Thông tin mô tả  28658 non-null  object 
 8   Diện tích        28658 non-null  float64
 9   Mức giá          28658 non-null  object 
 10  Mặt tiền         18348 non-null  objec

### Delete attribute columns

In [3]:
# Print the oldest and newest posting dates
print("Oldest posting date:", dtset['Ngày đăng'].min())
print("Newest posting date:", dtset['Ngày đăng'].max())

# Print the oldest and newest expiration dates
print("Oldest expiration date:", dtset['Ngày hết hạn'].min())
print("Newest expiration date:", dtset['Ngày hết hạn'].max())

Oldest posting date: 2024-02-04
Newest posting date: 2024-03-15
Oldest expiration date: 2024-03-04
Newest expiration date: 2024-04-11


Notice that the gap between the posting date and the expiration date is negligible (only about 1 month) => The range of house price fluctuations in that range will not be too large => Skip this section, drop the "Ngày đăng" and "Ngày hết hạn" columns

"Người đăng" feature has no value => drop 

In [4]:
# Print the initial dataset size
print("Initial dataset size:", dtset.shape)

# Drop specified columns
dtset.drop(['Ngày đăng', 'Ngày hết hạn', 'Người đăng'], axis=1, inplace=True)

# Print the dataset size after removing some attribute columns
print("Dataset size after removing some attribute columns:", dtset.shape)

Initial dataset size: (28658, 20)
Dataset size after removing some attribute columns: (28658, 17)


In [5]:
# Update the list of columns after removing some columns
dtset_columns_after = dtset.columns.tolist()

In [6]:
# Summary
# Initialize an empty list to store removed columns
delete_col_count = []

# Iterate through the original list of columns
for element in dtset_columns:
    # Check if the column is not in the updated list of columns
    if element not in dtset_columns_after:
        # If the column is not in the updated list, append it to the list of removed columns
        delete_col_count.append(element)

# Print the removed columns and their count
print("The removed attribute columns:", delete_col_count)
print("Total number of removed columns:", len(delete_col_count))

The removed attribute columns: ['Ngày đăng', 'Ngày hết hạn', 'Người đăng']
Total number of removed columns: 3


### Processing the "Địa chỉ" column

In [7]:
dtset['Địa chỉ'].unique()

array(['Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dương Nội, Hà Đông, Hà Nội',
       'Dự án An Khang Villa, Đường Tố Hữu, Phường La Khê, Hà Đông, Hà Nội',
       'Dự án Hado Charm Villas, Đường Đại lộ Thăng Long, Xã An Thượng, Hoài Đức, Hà Nội.',
       ..., 'Phường Hàng Bạc, Hoàn Kiếm, Hà Nội',
       'Đường Lương Ngọc Quyến, Phường Hàng Buồm, Hoàn Kiếm, Hà Nội',
       'Phố Hàng Dầu, Phường Hàng Bạc, Hoàn Kiếm, Hà Nội.'], dtype=object)

In [8]:
# Loop through the dataset and print full address information
for address in dtset['Địa chỉ']:
    print(address)

Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dương Nội, Hà Đông, Hà Nội
Dự án An Khang Villa, Đường Tố Hữu, Phường La Khê, Hà Đông, Hà Nội
Dự án Hado Charm Villas, Đường Đại lộ Thăng Long, Xã An Thượng, Hoài Đức, Hà Nội.
Phố Cầu Cốc, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội.
Dự án 671 Hoàng Hoa Thám, Đường Hoàng Hoa Thám, Phường Liễu Giai, Ba Đình, Hà Nội
Đường Xuân Đỉnh, Phường Xuân Đỉnh, Bắc Từ Liêm, Hà Nội
Đường Đình Thôn, Phường Mỹ Đình 1, Nam Từ Liêm, Hà Nội
Phố Từ Hoa, Phường Quảng An, Tây Hồ, Hà Nội
Phố Đại La, Phường Đồng Tâm, Hai Bà Trưng, Hà Nội
Phố Khâm Thiên, Phường Khâm Thiên, Đống Đa, Hà Nội
Dự án Hinode Royal Park, Đường Quốc lộ 32, Xã Di Trạch, Hoài Đức, Hà Nội
Phố Trần Cung, Phường Cổ Nhuế 1, Bắc Từ Liêm, Hà Nội
Phố Trung Kính, Phường Yên Hòa, Cầu Giấy, Hà Nội
Đường Trịnh Văn Bô, Phường Xuân Phương, Nam Từ Liêm, Hà Nội.
Dự án Roman Plaza, Đường Tố Hữu, Phường Đại Mỗ, Nam Từ Liêm, Hà Nội.
Đường Phương Canh, Phường Phương Canh, Nam Từ Liêm, Hà Nội.
Đường Đức Diễn, Phường Phú D

Add "Đường/Phố" column

In [9]:
# Create a list to store street information
streets = [np.nan for i in range(dtset.shape[0])]
print("Streets length before =", len(streets))  # "streets" is a new column named "Đường" extracted from the "Địa chỉ" column.
print("Streets sample before:", streets[0:3])

Streets length before = 28658
Streets sample before: [nan, nan, nan]


In [10]:
# Create a list to store street information
streets = []

# Loop through the dataset
for index in range(dtset.shape[0]):
    # Skip if the value of 'Địa chỉ' at index 'index' is NaN
    if pd.isnull(dtset['Địa chỉ'][index]):
        streets.append(np.nan)  # Append NaN if address is NaN
        continue
    
    # Split address by ", " to extract components
    address_components = dtset['Địa chỉ'][index].split(", ")

    street_found = False
    # Iterate over components to find street
    for component in address_components:
        if 'đường' in component.lower() or 'phố' in component.lower():
            # Remove 'đường' and 'phố' and leading/trailing whitespaces
            street = component.replace('Đường', '').replace('Phố', '').strip()
            streets.append(street)
            street_found = True
            break
    
    if not street_found:
        # If neither 'đường' nor 'phố' is found in any component, append NaN
        streets.append(np.nan)

In [11]:
print("Streets length after =", len(streets))
print("Streets sample after:", streets[0:3])

Streets length after = 28658
Streets sample after: ['Tố Hữu', 'Tố Hữu', 'Đại lộ Thăng Long']


In [12]:
streets

['Tố Hữu',
 'Tố Hữu',
 'Đại lộ Thăng Long',
 'Cầu Cốc',
 'Hoàng Hoa Thám',
 'Xuân Đỉnh',
 'Đình Thôn',
 'Từ Hoa',
 'Đại La',
 'Khâm Thiên',
 'Quốc lộ 32',
 'Trần Cung',
 'Trung Kính',
 'Trịnh Văn Bô',
 'Tố Hữu',
 'Phương Canh',
 'Đức Diễn',
 'Thụy Phương',
 'Võ Chí Công',
 'Nguyễn Đình Tứ',
 'Yên Lộ',
 'Ngọc Hà',
 'Tô Hiệu',
 '5',
 'Vũ Hữu',
 'Quốc lộ 32',
 'Phạm Hùng',
 'Nguyễn Đình Thi',
 'Âu Cơ',
 'Ngọc Thụy',
 'Doãn Kế Thiện',
 'Đền Lừ',
 'Kim Giang',
 'Hàng Bạc',
 'Nam Đuống',
 'Phùng Khoang',
 'Phạm Huy Thông',
 'Chiến Thắng',
 nan,
 'Bà Triệu',
 '422B',
 'Nguyên Xá',
 'Nguyễn Chánh',
 'Ngọc Thụy',
 'Kim Đồng',
 'Định Công',
 'Phạm Tuấn Tài',
 'Cổ Nhuế',
 'Nguyễn Xiển',
 'Đại Mỗ',
 'Trung Hòa',
 'Xuân Đỉnh',
 nan,
 'Tố Hữu',
 'Nhật Chiêu',
 'Bằng A',
 nan,
 'Nguyễn Văn Huyên',
 'Lạc Long Quân',
 nan,
 'Lê Trọng Tấn',
 'Kim Ngưu',
 'Trịnh Văn Bô',
 'Nguyễn Xiển',
 'Mễ Trì Hạ',
 'Hoàng Công',
 'Hoàng Quốc Việt',
 nan,
 'Di Trạch',
 'Tây Mỗ',
 'Vĩnh Ninh',
 '70',
 nan,
 'Hàm Nghi',


In [13]:
# Add the "Đường" column to the dataset based on the "streets" list:
dtset.insert(2, "Đường/Phố", streets, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (28658, 18)


,Tiêu đề,Địa chỉ,Đường/Phố,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


Add "Phường/Xã" column

In [14]:
ward = [np.nan for i in range(dtset.shape[0])]  # Create a list named "ward" with NaN values, with the length equal to the number of rows in the dataset.
print("Ward length before =", len(ward))  # "ward" is a new column named "Phường" extracted from the "Địa chỉ" column.
print("Ward sample before:", ward[0:3])
#ward_dictionary = {} # Contains exceptional cases that cannot immediately construct the "Phường" column.
ward_exception_list = []  # Contains exceptional cases in a list variable.

Ward length before = 28658
Ward sample before: [nan, nan, nan]


In [15]:
for index in range(dtset.shape[0]):
    if(type(dtset['Địa chỉ'][index]) == float):
        # Skip the current iteration if the value of 'Địa chỉ' at index 'index' is of type float.
        continue
    if 'Phường' not in dtset['Địa chỉ'][index] and 'Xã' not in dtset['Địa chỉ'][index] and 'Thị trấn' not in dtset['Địa chỉ'][index]:
        # If the value of 'Địa chỉ' at index 'index' does NOT contain the strings 'Phường', 'Xã', and 'Thị trấn':
        #ward_dictionary[index] = df['Địa chỉ'][index] # Note this down in ward_dictionary.
        ward_exception_list.append(index)  # Note the exception in ward_exception_list for deletion later.
        continue  # and do nothing.
    address = dtset['Địa chỉ'][index].split(", ")  # Example of an address, then:
    # "Đường Tố Hữu, Phường Dương Nội, Quận Hà Đông, Hà Nội" -> ['Đường Tố Hữu', 'Phường Dương Nội', 'Hà Đông', 'Hà Nội']
    for row in address:
        if 'Phường' in row or 'Xã' in row or 'Thị trấn' in row:
            # If the value of 'Địa chỉ' at index 'index' DOES contain the strings 'Phường', 'Xã', or 'Thị trấn':
            # Use the .strip() method to remove leading and trailing whitespaces.
            new_row = row.replace('Phường', '').replace('Xã', '').replace('Thị trấn', '').strip()
            #new_row = row.strip() # Do not remove 'Phường', 'Xã', or 'Thị trấn' if present.
            if new_row:
                ward[index] = new_row  # Save it as the last result to create a new column.
            else:
                ward[index] = np.nan  # If new_row is empty, assign 'NaN'
            continue
    #print(address)

In [16]:
print("Ward length after =", len(ward))
print("Ward sample after:", ward[0:3])

Ward length after = 28658
Ward sample after: ['Dương Nội', 'La Khê', 'An Thượng']


In [17]:
print("Total number of exceptional cases =", len(ward_exception_list))
#print("ward_dictionary length =", len(ward_dictionary))
#display(ward_dictionary)
print("Example ward_exception_list[0:10]:", ward_exception_list[0:10])
print("Example of exceptional value 1 (in the Dataset):", dtset['Địa chỉ'][ward_exception_list[0]])
print("Example of exceptional value 1 (in the ward variable):", ward[ward_exception_list[0]])

Total number of exceptional cases = 201
Example ward_exception_list[0:10]: [67, 132, 709, 711, 750, 839, 991, 1112, 1126, 1424]
Example of exceptional value 1 (in the Dataset): Dự án Vinhomes Riverside, Quận Long Biên, Hà Nội
Example of exceptional value 1 (in the ward variable): nan


In [18]:
ward

['Dương Nội',
 'La Khê',
 'An Thượng',
 'Tây Mỗ',
 'Liễu Giai',
 'Xuân Đỉnh',
 'Mỹ Đình 1',
 'Quảng An',
 'Đồng Tâm',
 'Khâm Thiên',
 'Di Trạch',
 'Cổ Nhuế 1',
 'Yên Hòa',
 'Xuân Phương',
 'Đại Mỗ',
 'Phương Canh',
 'Phú Diễn',
 'Thụy Phương',
 'Xuân La',
 'Xuân Đỉnh',
 'Yên Nghĩa',
 'Ngọc Hà',
 'Nguyễn Trãi',
 'Hải Bối',
 'Phương Liệt',
 'Tân Lập',
 'Mễ Trì',
 'Thụy Khuê',
 'Tứ Liên',
 'Ngọc Thụy',
 'Mai Dịch',
 'Hoàng Văn Thụ',
 'Đại Kim',
 'Hàng Bạc',
 'Đức Giang',
 'Trung Văn',
 'Ngọc Khánh',
 'Văn Quán',
 'Xuân La',
 'Nguyễn Trãi',
 'Vân Canh',
 'Minh Khai',
 'Yên Hòa',
 'Ngọc Thụy',
 'Giáp Bát',
 'Định Công',
 'Dịch Vọng Hậu',
 'Cổ Nhuế 2',
 'Hạ Đình',
 'Đại Mỗ',
 'Trung Hòa',
 'Xuân Đỉnh',
 'Việt Hưng',
 'Dương Nội',
 'Nhật Tân',
 'Hoàng Liệt',
 'Đại Thành',
 'Xuân Tảo',
 'Bưởi',
 'Phúc Đồng',
 'La Khê',
 'Thanh Lương',
 'Xuân Phương',
 'Tân Triều',
 'Mễ Trì',
 'Kiến Hưng',
 'Nghĩa Tân',
 nan,
 'Di Trạch',
 'Tây Mỗ',
 'Vĩnh Quỳnh',
 'Tây Mỗ',
 'Cự Khê',
 'Mỹ Đình 2',
 'Q

In [19]:
# Add the "Phường" column to the dataset based on the "ward" variable list:
dtset.insert(3, "Phường/Xã", ward, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (28658, 19)


,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


Add "Quận/Huyện" column

In [20]:
# Create a list named "district" with NaN values, with the length equal to the number of rows in the dataset.
district = [np.nan for i in range(dtset.shape[0])]
print("District length before =", len(district))  # "district" is a new column named "Quận" extracted from the "Địa chỉ" column.
print("District sample before:", district[0:3])

District length before = 28658
District sample before: [nan, nan, nan]


In [21]:
# List to store districts
district = []

# Loop through the dataset
for index in range(dtset.shape[0]):
    # Skip if the value of 'Địa chỉ' at index 'index' is NaN
    if pd.isnull(dtset['Địa chỉ'][index]):
        district.append(np.nan)
        continue
    
    # Split address by ", " to extract components
    address_components = dtset['Địa chỉ'][index].split(", ")

    district_found = False
    # Iterate over components to find district
    for component in address_components:
        if 'Quận' in component or 'Huyện' in component:
            # Remove 'Quận' or 'Huyện' and leading/trailing whitespaces
            district_name = component.replace('Quận', '').replace('Huyện', '').strip()
            district.append(district_name)
            district_found = True
            break
    
    if not district_found:
        # If neither 'Quận' nor 'Huyện' is found in any component, consider the last component as district
        district.append(address_components[-2].strip())  # Assuming the district is the second last component

In [22]:
print("District length after =", len(district))
print("District sample after:", district[0:3])
# print("Total number of exceptional cases =", len(district_exception_list))

District length after = 28658
District sample after: ['Hà Đông', 'Hà Đông', 'Hoài Đức']


In [23]:
district

['Hà Đông',
 'Hà Đông',
 'Hoài Đức',
 'Nam Từ Liêm',
 'Ba Đình',
 'Bắc Từ Liêm',
 'Nam Từ Liêm',
 'Tây Hồ',
 'Hai Bà Trưng',
 'Đống Đa',
 'Hoài Đức',
 'Bắc Từ Liêm',
 'Cầu Giấy',
 'Nam Từ Liêm',
 'Nam Từ Liêm',
 'Nam Từ Liêm',
 'Bắc Từ Liêm',
 'Bắc Từ Liêm',
 'Tây Hồ',
 'Bắc Từ Liêm',
 'Hà Đông',
 'Ba Đình',
 'Hà Đông',
 'Đông Anh',
 'Thanh Xuân',
 'Đan Phượng',
 'Nam Từ Liêm',
 'Tây Hồ',
 'Tây Hồ',
 'Long Biên',
 'Cầu Giấy',
 'Hoàng Mai',
 'Hoàng Mai',
 'Hoàn Kiếm',
 'Long Biên',
 'Nam Từ Liêm',
 'Ba Đình',
 'Hà Đông',
 'Tây Hồ',
 'Hà Đông',
 'Hoài Đức',
 'Bắc Từ Liêm',
 'Cầu Giấy',
 'Long Biên',
 'Hoàng Mai',
 'Hoàng Mai',
 'Cầu Giấy',
 'Bắc Từ Liêm',
 'Thanh Xuân',
 'Nam Từ Liêm',
 'Cầu Giấy',
 'Bắc Từ Liêm',
 'Long Biên',
 'Hà Đông',
 'Tây Hồ',
 'Hoàng Mai',
 'Quốc Oai',
 'Bắc Từ Liêm',
 'Tây Hồ',
 'Long Biên',
 'Hà Đông',
 'Hai Bà Trưng',
 'Nam Từ Liêm',
 'Thanh Trì',
 'Nam Từ Liêm',
 'Hà Đông',
 'Cầu Giấy',
 'Long Biên',
 'Hoài Đức',
 'Nam Từ Liêm',
 'Thanh Trì',
 'Nam Từ Liêm',


In [24]:
# Add the "Quận" column to the dataset based on the "district" variable list:
dtset.insert(4, "Quận/Huyện", district, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (28658, 20)


,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Quận/Huyện,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Hà Đông,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Hà Đông,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Hoài Đức,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


In [25]:
# Đếm số lượng giá trị NaN trong cột 'Quận/Huyện'
nan_district_count = dtset['Quận/Huyện'].isnull().sum()

# Đếm số lượng giá trị NaN trong cột 'Phường/Xã'
nan_ward_count = dtset['Phường/Xã'].isnull().sum()

# Đếm số lượng giá trị NaN trong cột 'Đường/Phố'
nan_street_count = dtset['Đường/Phố'].isnull().sum()

print("Số lượng giá trị NaN trong cột 'Quận/Huyện':", nan_district_count)
print("Số lượng giá trị NaN trong cột 'Phường/Xã':", nan_ward_count)
print("Số lượng giá trị NaN trong cột 'Đường/Phố':", nan_street_count)


Số lượng giá trị NaN trong cột 'Quận/Huyện': 0
Số lượng giá trị NaN trong cột 'Phường/Xã': 201
Số lượng giá trị NaN trong cột 'Đường/Phố': 3280


In [26]:
dtset['Quận/Huyện'].unique()

array(['Hà Đông', 'Hoài Đức', 'Nam Từ Liêm', 'Ba Đình', 'Bắc Từ Liêm',
       'Tây Hồ', 'Hai Bà Trưng', 'Đống Đa', 'Cầu Giấy', 'Đông Anh',
       'Thanh Xuân', 'Đan Phượng', 'Long Biên', 'Hoàng Mai', 'Hoàn Kiếm',
       'Quốc Oai', 'Thanh Trì', 'Thanh Oai', 'Gia Lâm', 'Mê Linh',
       'Chương Mỹ', 'Sơn Tây', 'Thạch Thất', 'Sóc Sơn', 'Thường Tín',
       'Phúc Thọ', 'Ứng Hòa', 'Hà Nội', 'Ba Vì',
       'liên thông phố Tạ Quang Bửu', 'Đường Ngõ', 'Phường Phú m Từ Liêm',
       'Phường Cầu Diễn', 'kinh doanh tốt', 'Vinh', 'Văn Giang'],
      dtype=object)

In [27]:
# In ra một số mẫu dữ liệu từ cột "Địa chỉ"
for address in dtset['Địa chỉ'].head(10):
    print(address)

Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dương Nội, Hà Đông, Hà Nội
Dự án An Khang Villa, Đường Tố Hữu, Phường La Khê, Hà Đông, Hà Nội
Dự án Hado Charm Villas, Đường Đại lộ Thăng Long, Xã An Thượng, Hoài Đức, Hà Nội.
Phố Cầu Cốc, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội.
Dự án 671 Hoàng Hoa Thám, Đường Hoàng Hoa Thám, Phường Liễu Giai, Ba Đình, Hà Nội
Đường Xuân Đỉnh, Phường Xuân Đỉnh, Bắc Từ Liêm, Hà Nội
Đường Đình Thôn, Phường Mỹ Đình 1, Nam Từ Liêm, Hà Nội
Phố Từ Hoa, Phường Quảng An, Tây Hồ, Hà Nội
Phố Đại La, Phường Đồng Tâm, Hai Bà Trưng, Hà Nội
Phố Khâm Thiên, Phường Khâm Thiên, Đống Đa, Hà Nội


In [28]:
keywords = ['liên thông phố Tạ Quang Bửu', 'Đường Ngõ', 'Phường Phú m Từ Liêm', 'Phường Cầu Diễn', 'kinh doanh tốt', 'Vinh', 'Văn Giang']

# Loop through the dataset
for index, address in enumerate(dtset['Địa chỉ']):
    # Skip if the value of 'Địa chỉ' at index 'index' is NaN
    if pd.isnull(address):
        continue
    
    # Check if any keyword is in the address
    for keyword in keywords:
        if keyword in address:
            print("Index:", index)
            print("Address:", address)
            print()
            break
    else:
        continue
    break


Index: 67
Address: Dự án Vinhomes Riverside, Quận Long Biên, Hà Nội



In [29]:
# Các từ khóa cần kiểm tra
keywords = ['Phường Cầu Diễn', 'Phường Phú m Từ Liêm', 
            'Vinh', 'Văn Giang', 'kinh doanh tốt', 'liên thông phố Tạ Quang Bửu']

# Lặp qua từng từ khóa và in ra các dòng thỏa mãn
for keyword in keywords:
    filtered_rows = dtset[dtset['Địa chỉ'].str.contains(keyword)]
    print(f"Dòng có địa chỉ chứa '{keyword}':")
    print(filtered_rows['Địa chỉ'])  # In chỉ cột 'Địa chỉ'
    print()


Dòng có địa chỉ chứa 'Phường Cầu Diễn':
936      Dự án Mỹ Đình I, Đường Hàm Nghi, Phường Cầu Di...
956      Dự án Vinhomes Gardenia, Đường Hàm Nghi, Phườn...
1129     Đường Hàm Nghi, Phường Cầu Diễn, Nam Từ Liêm, ...
1335     Dự án Mỹ Đình I, Đường Lưu Hữu Phước, Phường C...
1469     Dự án Vinhomes Gardenia, Đường Hàm Nghi, Phườn...
                               ...                        
26400    Đường Lưu Hữu Phước, Phường Cầu Diễn, Nam Từ L...
27304    Đường Hồ Tùng Mậu, Phường Cầu Diễn, Nam Từ Liê...
27457    Đường Hồ Tùng Mậu, Phường Cầu Diễn, Nam Từ Liê...
27863    Đường Hàm Nghi, Phường Cầu Diễn, Nam Từ Liêm, ...
28217    Đường Cầu Diễn, Phường Cầu Diễn, Nam Từ Liêm, ...
Name: Địa chỉ, Length: 198, dtype: object

Dòng có địa chỉ chứa 'Phường Phú m Từ Liêm':
13902    Đường Lê Quang Đạo, Phường Phú m Từ Liêm, Hà Nội
Name: Địa chỉ, dtype: object

Dòng có địa chỉ chứa 'Vinh':
67        Dự án Vinhomes Riverside, Quận Long Biên, Hà Nội
80       Dự án Vinhomes The Harmony, Đường Tuli

In [30]:
print("Địa chỉ của dòng số 67:", dtset['Địa chỉ'][67])
print("Địa chỉ của dòng số 80:", dtset['Địa chỉ'][80])
print("Địa chỉ của dòng số 110:", dtset['Địa chỉ'][110])
print("Địa chỉ của dòng số 132:", dtset['Địa chỉ'][132])
print("Địa chỉ của dòng số 135:", dtset['Địa chỉ'][135])


Địa chỉ của dòng số 67: Dự án Vinhomes Riverside, Quận Long Biên, Hà Nội
Địa chỉ của dòng số 80: Dự án Vinhomes The Harmony, Đường Tulip 4, Phường Việt Hưng, Long Biên, Hà Nội
Địa chỉ của dòng số 110: Dự án Vinhomes Ocean Park Gia Lâm, Xã Dương Xá, Gia Lâm, Hà Nội
Địa chỉ của dòng số 132: Dự án Vinhomes Ocean Park Gia Lâm, Huyện Gia Lâm, Hà Nội
Địa chỉ của dòng số 135: Dự án Vinhomes Green Bay Mễ Trì, Đường Đại lộ Thăng Long, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.


In [31]:
filtered_rows1 = dtset[dtset['Địa chỉ'].str.contains('kinh doanh tốt')]
filtered_rows1['Địa chỉ']
print("Địa chỉ của dòng số 22196:", dtset['Địa chỉ'][22196])

Địa chỉ của dòng số 22196: Bán nhà, Yên Nghĩa Hà Đông, Xây mới, thiết kế đẹp, thuận tiện giao thông, kinh doanh tốt, gần công viên..


In [32]:
filtered_rows2 = dtset[dtset['Địa chỉ'].str.contains('Đường Ngõ')]
filtered_rows2['Địa chỉ']

# print("Địa chỉ của dòng số 22196:", dtset['Địa chỉ'][22196])

11134    Đường Ngõ Huyện, Phường Hàng Trống, Hoàn Kiế...
16150    Đường Ngõ Nội Miếu, Phường Hàng Buồm, Hoàn K...
23525    Đường Ngõ Bảo Khánh, Phường Hàng Trống, Hoàn...
Name: Địa chỉ, dtype: object

In [33]:
print(len(dtset['Đường/Phố'].unique()))
print("Số lượng giá trị trong cột 'Đường/Phố':", len(dtset['Đường/Phố']))

1256
Số lượng giá trị trong cột 'Đường/Phố': 28658


In [34]:
print(len(dtset['Phường/Xã'].unique()))
print("Số lượng giá trị trong cột 'Phường/Xã':", len(dtset['Phường/Xã']))

366
Số lượng giá trị trong cột 'Phường/Xã': 28658


In [35]:
print(len(dtset['Quận/Huyện'].unique()))
print("Số lượng giá trị trong cột 'Phường/Xã':", len(dtset['Quận/Huyện']))

36
Số lượng giá trị trong cột 'Phường/Xã': 28658


In [37]:
# # Create a list to store street information
# streets1 = []

# # Loop through the dataset
# for index in range(dtset.shape[0]):
#     # Skip if the value of 'Địa chỉ' at index 'index' is NaN
#     if pd.isnull(dtset['Địa chỉ'][index]):
#         streets1.append(np.nan)  # Append NaN if address is NaN
#         continue
    
#     # Split address by ", " to extract components
#     address_components = dtset['Địa chỉ'][index].split(", ")

#     street_found = False
#     # Iterate over components to find street
#     for component in address_components:
#         if 'đường' in component.lower() or 'phố' in component.lower():
#             # Remove 'đường' and 'phố' and leading/trailing whitespaces
#             street = component.replace('Đường', '').replace('Phố', '').strip()
#             streets1.append(street)
#             street_found = True
#             break
    
#     if not street_found:
#         # If neither 'đường' nor 'phố' is found in any component, append NaN
#         streets1.append(np.nan)

# # Print the extracted street information
# print(streets1)
# # Kiểm tra số lượng giá trị đã trích xuất và lưu trữ trong danh sách streets
# print("Số lượng giá trị đã trích xuất:", len(streets))

# unique_values_count = len(set(streets1))
# print("Số giá trị duy nhất trong danh sách pho_duong là:", unique_values_count)

['Tố Hữu', 'Tố Hữu', 'Đại lộ Thăng Long', 'Cầu Cốc', 'Hoàng Hoa Thám', 'Xuân Đỉnh', 'Đình Thôn', 'Từ Hoa', 'Đại La', 'Khâm Thiên', 'Quốc lộ 32', 'Trần Cung', 'Trung Kính', 'Trịnh Văn Bô', 'Tố Hữu', 'Phương Canh', 'Đức Diễn', 'Thụy Phương', 'Võ Chí Công', 'Nguyễn Đình Tứ', 'Yên Lộ', 'Ngọc Hà', 'Tô Hiệu', '5', 'Vũ Hữu', 'Quốc lộ 32', 'Phạm Hùng', 'Nguyễn Đình Thi', 'Âu Cơ', 'Ngọc Thụy', 'Doãn Kế Thiện', 'Đền Lừ', 'Kim Giang', 'Hàng Bạc', 'Nam Đuống', 'Phùng Khoang', 'Phạm Huy Thông', 'Chiến Thắng', nan, 'Bà Triệu', '422B', 'Nguyên Xá', 'Nguyễn Chánh', 'Ngọc Thụy', 'Kim Đồng', 'Định Công', 'Phạm Tuấn Tài', 'Cổ Nhuế', 'Nguyễn Xiển', 'Đại Mỗ', 'Trung Hòa', 'Xuân Đỉnh', nan, 'Tố Hữu', 'Nhật Chiêu', 'Bằng A', nan, 'Nguyễn Văn Huyên', 'Lạc Long Quân', nan, 'Lê Trọng Tấn', 'Kim Ngưu', 'Trịnh Văn Bô', 'Nguyễn Xiển', 'Mễ Trì Hạ', 'Hoàng Công', 'Hoàng Quốc Việt', nan, 'Di Trạch', 'Tây Mỗ', 'Vĩnh Ninh', '70', nan, 'Hàm Nghi', 'Dương Quảng Hàm', 'Tả Thanh Oai', 'Trần Nguyên Đán', nan, 'Đại Đồng', 'P

In [38]:
# Lọc ra các dòng có giá trị NaN trong cột 'Đường/Phố'
rows_with_nan_street = dtset.loc[dtset['Đường/Phố'].isnull()]

# In ra các dòng đó
print(rows_with_nan_street)


                                                 Tiêu đề  \
38         Nhà đẹp, hiếm Tây Hồ, ngõ ô tô tránh. 3.xx tỷ   
52     Siêu biệt thự Việt Hưng, nhà đẹp, ở luôn, tiện...   
56     Bán nhà riêng 65m2 xây 3T mới tại Đại Thành - ...   
59     Bán đất tặng nhà 3 tầng 95m2 khu phân lô Phúc ...   
67     Chính chủ bán BT đơn lập góc hoàn thiện full đ...   
...                                                  ...   
28403  Cần bán toà nhà DT 180m2 lô góc 9 tầng tại tru...   
28467  Chính chủ bán liền kề Vạn Phúc - Hà Đông, 7 tầ...   
28571  Bán nhà 5 tầng mới diện tích 50m2, MT 5m, đườn...   
28575  Bán tòa nhà mặt phố quận Hai Bà Trưng - phường...   
28610  Bán gấp nhà hơn 100m2 mặt Hồ Gươm đặc biệt nhà...   

                                                 Địa chỉ Đường/Phố  \
38                        Phường Xuân La, Tây Hồ, Hà Nội       NaN   
52     Dự án Việt Hưng, Phường Việt Hưng, Long Biên, ...       NaN   
56                        Xã Đại Thành, Quốc Oai, Hà Nội       NaN   

In [39]:
print("Địa chỉ của dòng số 52:", dtset['Địa chỉ'][52])

Địa chỉ của dòng số 52: Dự án Việt Hưng, Phường Việt Hưng, Long Biên, Hà Nội
